<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: Unable to read current working directory: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: Unable to read current working directory: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'Walmart-Recruiting-Store-Sales-Forecasting': No such file or directory
[Errno 2] No such file or directory: 'Walmart-Recruiting-Store-Sales-Forecasting'
/content/Walmart-Recruiting-Store-Sales-Forecasting


In [3]:
from google.colab import userdata
!pip install -r requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 67.2 MB/s eta 0:00:00
Setting up Kaggle credentials...
Ensuring data directory exists at 'data/'...
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 621MB/s]
Unzipping files...
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  features.csv.zip
  inflating: features.csv            
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
Data downloaded and extracted successfully to 'data/'.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [51]:
from feature_engineering.time_features import FeatureAdder
FeatureAdder(add_dummy_date=True,start_date=pd.Timestamp('2010-02-05')).fit_transform(train).columns

TypeError: FeatureAdder.__init__() got an unexpected keyword argument 'add_dummy_date'

## **Merge tables and split data**

In [6]:
merged_train = pd.merge(train,stores,on='Store',how='left').merge(features,how='left',on=['Store','Date','IsHoliday'])

In [7]:
from src.time_series_split import TimeSeriesSplit
from src.config import SPLIT_DATE
merged_train.Date = pd.to_datetime(merged_train.Date)
x_train, x_val = TimeSeriesSplit(SPLIT_DATE).split(merged_train)
y_train = x_train.pop('Weekly_Sales')
y_val = x_val.pop('Weekly_Sales')

# **Time series features**

In [8]:
from feature_engineering.time_features import FeatureAdder

# deal with **NaN** values

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class NaImputer(BaseEstimator, TransformerMixin):
  def __init__(self, strategy='mean'):
    self.strategy = strategy
    self.imputer = SimpleImputer(strategy=strategy)
    self.na_cols = []

  def fit(self, X, y=None):
    self.na_cols = [col for col in X.columns if X[col].isna().sum() > 0]
    self.imputer.fit(X[self.na_cols])
    return self

  def transform(self, X, y=None):
    x_copy = X.copy()
    x_copy[self.na_cols] = self.imputer.transform(x_copy[self.na_cols])
    return x_copy

# **Cat2Num**

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class Cat2Num(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    x_copy = X.copy()
    x_copy.pop('Date')
    x_copy.IsHoliday = x_copy.IsHoliday.astype(int)
    x_copy.Type = x_copy.Type.astype('category').cat.codes
    return x_copy

# **XGboost**

In [11]:
from xgboost import DMatrix
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from src.utils import wmae

In [146]:
from src.cross_validation import manual_model_search

pipeline = Pipeline([
    ('adder', FeatureAdder()),
    ('imputer', NaImputer()),
    ('cat2num', Cat2Num()),
    ('model', XGBRegressor(
        n_estimators=1000,
        learning_rate=0.1,
        max_depth=7,
        reg_lambda=3,
        min_split_loss=100,
        objective='reg:squarederror',
        random_state=42,
    ))
  ])


param_grid = {
    'model__n_estimators': [1000],
    'model__learning_rate': [0.1],
    'model__max_depth': [7],
    'model__reg_lambda': [3],
    'model__min_split_loss': [100],
}


metric_kwargs = {
    'is_holiday': x_val['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=pipeline,
    param_grid=param_grid,
    X_train=x_train,
    y_train=y_train,
    X_valid=x_val,
    y_valid=y_val,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'model__n_estimators': 1000, 'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__reg_lambda': 3, 'model__min_split_loss': 1000} -> Score: 2901.4920

Best Params: {'model__n_estimators': 1000, 'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__reg_lambda': 3, 'model__min_split_loss': 1000}
Best Validation Score: 2901.492014852073


In [12]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('adder', FeatureAdder()),
    ('imputer', NaImputer()),
    ('cat2num', Cat2Num()),
    ('model', XGBRegressor(
        n_estimators=1000,
        learning_rate=0.1,
        max_depth=7,
        reg_lambda=3,
        min_split_loss=100,
        objective='reg:squarederror',
        random_state=42,
    ))
  ])

model = pipeline.fit(x_train, y_train)

y_train_predict = model.predict(x_train)
y_val_predict = model.predict(x_val)

train_score = wmae(y_train, y_train_predict,x_train['IsHoliday'].to_list())
val_score = wmae(y_val, y_val_predict,x_val['IsHoliday'].to_list())
print(f"Train wmae: {train_score}, Val wmae: {val_score}")

Train wmae: 1578.84506300633, Val wmae: 2848.694796243226


# **upload to WanDB**

In [36]:
! wandb login

wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [40]:
import wandb
import joblib

joblib.dump(model, "xgb_pipeline.pkl")
wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="xgboost:run1")

wandb.config.update({
    'merge1' : 'train, store, how=left, on=Store',
    'merge2' : 'train, features, how=left, on=Store, Date, IsHoliday',
    'merged_tables' : ['train','stores','features'],
    'time_features' : [
        'DateDummy', 'Month', 'Year',
        'WeekOfYear', 'Is_Christmas', 'Is_LaborDay', 'Is_Thanksgiving',
        'Is_SuperBowl', 'Days_until_next_holiday', 'Days_since_last_holiday',
        'week_sin', 'week_cos', 'month_sin', 'month_cos',
        'Days_until_next_Christmas', 'Days_since_last_Christmas',
        'Days_until_next_LaborDay', 'Days_since_last_LaborDay',
        'Days_until_next_Thanksgiving', 'Days_since_last_Thanksgiving',
        'Days_until_next_SuperBowl', 'Days_since_last_SuperBowl'
    ],
    'score_metric' : 'WMAE',
    'score_policy' : {
        'weight on holidays' : 5,
        'weight on non_holidays' : 1
    },
    'model' : 'Xgboost',
    'n_estimators' : 1000,
    'learning_rate' : 0.1,
    'max_depth' : 7,
    'reg_lambda' : 3,
    'min_split_loss' : 100,
    'objective' : 'reg:squarederror',
})

wandb.log({
    'train_wmae': train_score,
    'val_wmae': val_score
})


artifact = wandb.Artifact(
    name="xgb_pipeline",
    type="model",
    description="XGBoost pipeline with Date engineering and imputing"
)

artifact.add_file("xgb_pipeline.pkl")
wandb.log_artifact(artifact)

wandb.finish()

train_wmae,▁
val_wmae,▁
train_wmae,1578.84506
val_wmae,2848.6948


# **Load model from wandb**

In [42]:
import wandb
import joblib

# Resume or init run
run = wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="xgboost:run1")

# Download the artifact
artifact = run.use_artifact('MLBeasts/Walmart Recruiting - Store Sales Forecasting/xgb_pipeline:latest', type='model')
artifact_dir = artifact.download()

# Load the model
model = joblib.load(f"{artifact_dir}/xgb_pipeline.pkl")

wandb:   1 of 1 files downloaded.  


In [46]:
test = pd.read_csv(TEST_PATH)
merged_test = test.merge(stores,on='Store',how='left').merge(features,how='left',on=['Store','Date','IsHoliday'])
model.predict(merged_test)

ValueError: feature_names mismatch: ['Store', 'Dept', 'IsHoliday', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'DateDummy', 'Month', 'Year', 'WeekOfYear', 'Is_SuperBowl', 'Is_Thanksgiving', 'Is_Christmas', 'Is_LaborDay', 'Days_until_next_holiday', 'Days_since_last_holiday', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 'Days_until_next_SuperBowl', 'Days_since_last_SuperBowl', 'Days_until_next_Thanksgiving', 'Days_since_last_Thanksgiving', 'Days_until_next_Christmas', 'Days_since_last_Christmas', 'Days_until_next_LaborDay', 'Days_since_last_LaborDay'] ['Store', 'Dept', 'IsHoliday', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Month', 'Year', 'WeekOfYear', 'Is_SuperBowl', 'Is_Thanksgiving', 'Is_Christmas', 'Is_LaborDay', 'Days_until_next_holiday', 'Days_since_last_holiday', 'week_sin', 'week_cos', 'month_sin'